# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:

"""
THIS IS AN ADDED CELL FOR MAINTENANCE PURPOSES
THIS CHECK SOLVES "InvalidArgumentError" 
WHEN GPU AND UPDATED VERSION OF TENSORFLOW IS PRESENT!!!
"""
###Verifying AWS is using GPU and not CPU

from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


In [2]:
import helper

# Load English data
english_sentences = helper.load_data('small_vocab_en')
# Load French data
french_sentences = helper.load_data('small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [3]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [4]:
import collections

english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
In order for a Neural Network to understand its text Data Input and make Predictions, we first convert Text into Sequences of Integers, then add Padding using these pre-processing methods:
1. Tokenize the Words into IDs
    - Text data like "dog" is a Sequence of ASCII character encodings. Neural Networks are a series of Multiplication and Addition operations, so Input Data must be number(s).
2. Add Padding (converts all Sequences to same length).

Note: Text data is NOT used as Input to the Model

- Character IDs Definition - Convert each character into number. Used for character level models that generate text predictions for each character 
- Word ID Definition - convert each word into number. Word Level Models uses Word IDs that generate text predictions for each word. Word Level Models are lower complexity so tend to learn better.

### Tokenize (IMPLEMENTATION)

We will use Word Level Models and convert each sentence into a Sequence of Words IDs using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function to tokenize `english_sentences` and `french_sentences` below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [5]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(text_sentences):
    """
    Tokenize sentences
    :param sentences: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized sentences data, tokenizer used to tokenize sentences)
    """
    sentence_lengths = []
    for sentence in text_sentences:
        sentence_lengths.append(len(set(sentence.split())))
    max_sentence_length = max(sentence_lengths)

    # instance of tokenizer to prepare text for use in Neural Network models
    tokenizer = Tokenizer()
    # list of input texts for training phase to train on
    tokenizer.fit_on_texts(text_sentences)

    # list of sequences per input text
    tokenized_data = tokenizer.texts_to_sequences(text_sentences)

    return tokenized_data, tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized_sequences, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized_sequences)):
    print('Sequence {} from given Sentences'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'my': 12, 'quick': 2, 'jove': 11, 'jumps': 6, 'over': 7, 'short': 20, 'of': 14, 'by': 10, 'brown': 4, 'study': 13, 'the': 1, 'this': 18, 'lexicography': 15, 'lazy': 8, 'fox': 5, 'sentence': 21, 'dog': 9, 'won': 16, 'a': 3, 'is': 19, 'prize': 17}

Sequence 1 from given Sentences
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 from given Sentences
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 from given Sentences
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When Batching the Sequence of Word IDs together, each Sequence must be same length.  Since sentences are dynamic in length, we can add Padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [6]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(text_tokenized_sequences, length=None):
    """
    Pad text_tokenized_sequences
    :param text_tokenized_sequences: List of sequences.
    :param length: Length to pad the sequence to.  
      If None, use length of longest sequence in text_tokenized_sequences.
    :return: Padded numpy array of sequences
    """
    padded_sequences = pad_sequences(text_tokenized_sequences, # list of int
                                     maxlen=length, 
                                     padding='post') # pad after each sequence 'post'
    return padded_sequences
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized_sequences)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized_sequences, test_pad)):
    print('Sequence {} in text_tokenized_sequences'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in text_tokenized_sequences
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in text_tokenized_sequences
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in text_tokenized_sequences
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Building neural network architecture is the focus (so creating a pre-process pipeline is out of scope).  Instead, the implementation of `preprocess` function is provided.

In [7]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function 
    # requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, \
english_tokenizer, french_tokenizer = \
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you'll Experiment with various Neural Network Architectures.
You'll begin training 4x Simple Architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Word Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

Then you'll construct a Deeper Architecture that is designed to outperform all four models.

### Word IDs back to Text
The Neural Network will translate Input to Word IDs, which isn't the correct final form desired.  Since we want French translation, the function `logits_to_text` bridges the gap between the logits from the Neural Network to the French translation.  Use this function to better understand the Output of the Neural Network.

In [8]:
def logits_to_text(logits, tokenizer):
    """
    Convert Logits from a Neural Network into Text using Tokenizer
    :param logits: Logits from a Neural Network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the Text of the Logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] ='<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [9]:

from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    inputs = Input(shape=input_shape[1:])
    x = GRU(512, return_sequences=True)(inputs)
    x = TimeDistributed(Dense(french_vocab_size, activation="relu"))(x)
    outputs = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
simple_rnn_model.summary()

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 512)           789504    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           176472    
_________________________________________________________________
activation_2 (Activation)    (None, 21, 344)           0         
Total params: 965,976
Trainable params: 965,976
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 39s - loss: 2.8076 - acc: 0.5055 - val_loss: nan - val_acc: 0.5729
Epoch 2/10
110288/110288 [==============================] - 38s - loss: 2.1882 - acc: 0.58

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
Previously we converted Words into Word IDs, however Word Embeddings provide better representation of Words. Word Embeddings are a Vector representation of the Word that is close to similar words in n-dimensional space, where the n represents the size of the Word Embedding vectors.

In this model, you'll create a RNN model using embedding.

In [10]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inputs = Input(shape=input_shape[1:])
    x = Embedding(english_vocab_size, output_sequence_length)(inputs)
    x = GRU(512, return_sequences=True)(x)
    x = TimeDistributed(Dense(french_vocab_size, activation="relu"))(x)
    outputs = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# TODO: Train the neural network
embedding_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
embedding_rnn_model.summary()
embedding_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(logits_to_text(simple_rnn_model.predict(tmp_x)[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 21, 21)            4179      
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 512)           820224    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 344)           176472    
_________________________________________________________________
activation_4 (Activation)    (None, 21, 344)           0         
Total params: 1,000,875
Trainable params: 1,000,875
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 40s - loss: 3.3714 - 

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [11]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inputs = Input(shape=input_shape[1:])
    x = Bidirectional(GRU(512, return_sequences=True))(inputs)
    x = TimeDistributed(Dense(french_vocab_size, activation="relu"))(x)
    outputs = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    
    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.summary()
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 21, 1024)          1579008   
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 344)           352600    
_________________________________________________________________
activation_5 (Activation)    (None, 21, 344)           0         
Total params: 1,931,608
Trainable params: 1,931,608
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 39s - loss: 2.8715 - acc: 0.4937 - val_loss: nan - val_acc: 0.5539
Epoch 2/10
110288/110288 [==============================] - 38s - loss: 2.3254 - acc: 

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [13]:
from keras.layers import RepeatVector

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)
    inputs = Input(shape=input_shape[1:])
    x = Embedding(english_vocab_size, output_sequence_length)(inputs)
    x = Bidirectional(GRU(512))(x)
    x = RepeatVector(output_sequence_length)(x)
    x = Bidirectional(GRU(512, return_sequences=True))(x)
    x = TimeDistributed(Dense(french_vocab_size, activation="relu"))(x)
    outputs = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)

    learning_rate = 0.001
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])    
    model.summary()
    return model
tests.test_model_final(model_final)

print('Final Model Loaded')

(137861, 15) 21 199 344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 15, 21)            4179      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              1640448   
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 1024)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 21, 1024)          4721664   
_________________________________________________________________
time_distributed_7 (TimeDist (None, 21, 344)           352600    
_________________________________________________________________
activation_7 (Activation)    (None, 21, 344)        

## Prediction (IMPLEMENTATION)

In [14]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using model_final
    model = model_final(
        x.shape, #tmp_x.shape, # x.shape,
        y.shape[1], # preproc_french_sentences.shape[1],
        len(x_tk.word_index), # len(english_tokenizer.word_index),
        len(y_tk.word_index), # len(french_tokenizer.word_index)
    )
    
    model.fit(
        x, # tmp_x, # x
        y, # preproc_french_sentences, 
        batch_size=1024, 
        epochs=10, 
        validation_split=0.2
    )

    # Print prediction(s)
    print(logits_to_text(model.predict(x[:1])[0], y_tk))

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

(137861, 15) 21 199 344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 21)            4179      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1024)              1640448   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 1024)          0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 21, 1024)          4721664   
_________________________________________________________________
time_distributed_8 (TimeDist (None, 21, 344)           352600    
_________________________________________________________________
activation_8 (Activation)    (None, 21, 344)        

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.